In [121]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import datetime as dt
from webdriver_manager.chrome import ChromeDriverManager
import csv

In [122]:
# Set up Splinter - set the executable path and initialize a browser:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/102.0.5005.61/chromedriver_win32.zip
Driver has been 

In [123]:

def scrapeSite(character_name):
    table_head = []
    table_data = []
    url = f'https://pokemondb.net/pokedex/{character_name}'
    browser.visit(url)
    html = browser.html
    html_soup = soup(html, 'html.parser')
    get_table_data = html_soup.find('div', class_='sv-tabs-panel-list')
    #
    # GET Pokédex data / Vitals:
    pokedex_data_table = get_table_data.find(class_='vitals-table')
    table_body = pokedex_data_table.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        th = row.find_all('th')
        td = row.find_all('td')
        th = [x.text.strip() for x in th]
        td = [x.text.strip() for x in td]  
        table_head.append(th)
        table_data.append(td)
    # Get Training Data
    training = get_table_data.find(class_='grid-col span-md-12 span-lg-4')
    training_data = training.find_all(class_='vitals-table')[0]
    table_body = training_data.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        th=row.find_all('th')
        td=row.find_all('td')
        th=[x.text.strip() for x in th]
        td=[x.text.strip() for x in td]  
        table_head.append(th)
        table_data.append(td)
    # Breeding Data:
    breeding = training.find_all(class_='vitals-table')[1]
    table_body = breeding.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        th=row.find_all('th')
        td=row.find_all('td')
        th=[x.text.strip() for x in th]
        td=[x.text.strip() for x in td]  
        table_head.append(th)
        table_data.append(td)

    # Create DataFrame of Collected Data and Save Results:
    pokedex_head_df = pd.DataFrame(table_head)
    pokedex_data_df = pd.DataFrame(table_data)
    pokedex_table_df = pd.concat([pokedex_head_df, pokedex_data_df], axis=1).transpose()
    pokedex_table_df.to_csv(f'../scraped/Characters/{character_name}_pokedex_table_df.csv', encoding='utf-8', index=False, header=False)



In [124]:
# Character Names:
characters1 = pd.read_csv('../scraped/list_names.csv')
characters = pd.DataFrame(characters1)
characters_list = characters['Name']
characters_list = characters_list.tolist()

In [125]:
# RUN Script to gather all characters data:
for x in characters_list:
    scrapeSite(x)


In [2]:
# MERGE ALL FILES INTO A SINGLE CSV:
# https://stackoverflow.com/questions/45211681/add-column-to-csv-file-and-insert-filename-in-that-column
#import glob
#import csv
#directory = "../scraped/Characters/"
#with open('../scraped/AllCharactersMerged.csv', 'w', newline='') as f_output:
#    csv_output = csv.writer(f_output)
#
#    for filename in glob.glob(directory + '*df.csv'):
#        with open(filename, newline='') as f_input:
#            csv_input = csv.reader(f_input)
#
#            for row in csv_input:
#                row.insert(0, filename)
#                csv_output.writerow(row)